# 1. Import and read the data set

In [5]:
# [1] Import all the necessary packages:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import statsmodels.api as sm 
# Note: Indicates situations that aren’t necessarily exceptions.
import warnings  
warnings.filterwarnings("ignore")

# [2] Read the data file with Pandas.
df = pd.read_csv("Customer_data.csv")  

# [3] Sense-check the data.
df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Age            41188 non-null  int64  
 1   Occupation     41188 non-null  object 
 2   Status         41188 non-null  object 
 3   Edu            41188 non-null  object 
 4   House          41188 non-null  object 
 5   Loan           41188 non-null  object 
 6   Comm           41188 non-null  object 
 7   Month          41188 non-null  object 
 8   DOW            41188 non-null  object 
 9   Duration       41188 non-null  int64  
 10  Campaign       41188 non-null  int64  
 11  PosDays        41188 non-null  int64  
 12  Last_out       41188 non-null  object 
 13  Var_rate       41188 non-null  float64
 14  Price_idx      41188 non-null  float64
 15  Conf_idx       41188 non-null  float64
 16  Month_rate     41188 non-null  float64
 17  Quarterly_emp  41188 non-null  float64
 18  Target

Here we can see that there are 19 columns, 41,188 rows, and three different data types: integers, floats, and objects. When we compare the output to the BLR output, the results should be the same.



# 2. Clean the data

As with the BLR we need to update the Edu column to eliminate any periods. The same code snippet and explanation applies to the SVM model as with the BLR model.

In [6]:
# Update all the details of the education column:
df.loc[df['Edu'].str.contains('basic'),'Edu' ] = 'pre-school'
df.loc[df['Edu'].str.contains('university'),'Edu' ] = 'uni'
df.loc[df['Edu'].str.contains('high'),'Edu' ] = 'high-school'
df.loc[df['Edu'].str.contains('professional') ,'Edu'] = 'masters'
df.loc[df['Edu'].str.contains('illiterate'),'Edu' ] = 'other'
df.loc[df['Edu'].str.contains('unknown'),'Edu' ] = 'other'

# Display all the unique values/check changes.
df['Edu'].unique()  


array(['pre-school', 'other', 'uni', 'high-school', 'masters'],
      dtype=object)

# 3. Create dummy variables

The next step is to create the dummy variables (to account for the effects of one or more nominal-scale variables on the dependent variable). The process, explanations and code snippet are the same as with the BLR model. 

Do you remember and understand the code snippet to convert categorical variables to dummy variables? Try to write the code and then select the tab to reveal the full code snippet with comments.

In [3]:
# Convert categorical variables to dummy variables:
cat_vars=['Occupation','Status','Edu','House','Loan','Comm','Month','DOW','Last_out']

# Specify what needs to apply to all the variables
for var in cat_vars: 
    # Specify details of the categorical list
    cat_list = pd.get_dummies(df[var], prefix=var) 
    # Indicate the joining of the DataFrames
    df1=df.join(cat_list) 
    # Set old the DataFrame with new df with dummy values
    df=df1 
    
    cat_vars=['Occupation','Status','Edu','House','Loan','Comm','Month','DOW','Last_out']
    
# Set the temporary DataFrame and add values
df_vars=df.columns.values.tolist() 
# Indicate what columns are kept
to_keep=[i for i in df_vars if i not in cat_vars] 
    
# Define a new DataFrame
df_fin=df[to_keep] 

# Print the column
df_fin.columns.values 

array(['Age', 'Duration', 'Campaign', 'PosDays', 'Var_rate', 'Price_idx',
       'Conf_idx', 'Month_rate', 'Quarterly_emp', 'Target',
       'Occupation_admin.', 'Occupation_blue-collar',
       'Occupation_entrepreneur', 'Occupation_housemaid',
       'Occupation_management', 'Occupation_retired',
       'Occupation_self-employed', 'Occupation_services',
       'Occupation_student', 'Occupation_technician',
       'Occupation_unemployed', 'Occupation_unknown', 'Status_divorced',
       'Status_married', 'Status_single', 'Status_unknown',
       'Edu_high-school', 'Edu_masters', 'Edu_other', 'Edu_pre-school',
       'Edu_uni', 'House_no', 'House_unknown', 'House_yes', 'Loan_no',
       'Loan_unknown', 'Loan_yes', 'Comm_cellular', 'Comm_telephone',
       'Month_apr', 'Month_aug', 'Month_dec', 'Month_jul', 'Month_jun',
       'Month_mar', 'Month_may', 'Month_nov', 'Month_oct', 'Month_sep',
       'DOW_fri', 'DOW_mon', 'DOW_thu', 'DOW_tue', 'DOW_wed',
       'Last_out_failure', 'Last_out

In [7]:
features = ['Occupation', 'Status', 'Edu', 'House', 'Loan', 'Comm', 'Month',
       'DOW', 'Last_out']

df_fin = pd.get_dummies(df, columns = features)

# 4. Balance the data

In [4]:
# [1] Create the DataFrame to use as ‘df_fin’ and replace missing values with zero:
df_fin = df_fin.fillna(0)

# [2] Specify only the necessary columns for BLR: 
nec_cols = [ 'Status_divorced', 'Status_married',
            'Status_single', 'Status_unknown', 
            'Edu_high-school', 'Edu_masters', 
            'Edu_other', 'Edu_pre-school', 
            'Edu_uni', 'House_no', 'House_unknown',
            'House_yes', 'Loan_no', 'Loan_unknown', 
            'Loan_yes', 'DOW_fri', 'DOW_mon']

# [3a] Set the independent variable.
X = df_fin[nec_cols]  
# [3b] Set the dependent variable.
y = df_fin.loc[:, df_fin.columns == 'Target']   

# [4] Create a new DataFrame and 
# [4a] apply SMOTE as the target variable is not balanced.
os = SMOTE(random_state=0) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# [5] Specify column values.
columns = X_train.columns   

# [6] Specify the new data sets.   
os_data_X,os_data_y=os.fit_resample(X_train, y_train) 

# [7] Create two DataFrames for X and one for y:
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['Target'])

# [8] Print the DataFrame.
print("length of oversampled data is ",len(os_data_X)) 

os_data_y

length of oversampled data is  51134


Target
0           0
1           0
2           0
3           0
4           0
...       ...
51129       1
51130       1
51131       1
51132       1
51133       1

[51134 rows x 1 columns]

In [8]:
# Determine if values in a column are balanced.
os_data_y['Target'].value_counts()  

0    25567
1    25567
Name: Target, dtype: int64

# 5. Build and apply the SVM

In [11]:
# [1] Import the ‘svm’ package from sklearn. 
from sklearn import svm  
# [2] Import the ‘confusion_matrix’ package.
from sklearn.metrics import confusion_matrix  

# [3] Create an svm classifier using [3a] a linear kernel.
clf = svm.SVC(kernel='linear', gamma='scale')  

# [4] Train the model using the training sets.
clf.fit(os_data_X, os_data_y)  

# [5] Predict the response for the test data set.
y_pred = clf.predict(X_test)  

# 6. Check the accuracy of the model

Finally, the model was built and fitted to the data set. Let’s prepare the confusion matrix and accuracy report to determine how the SVM and BLR models compare. However, there is another way to calculate the accuracy, precision, and recall of the model. 

In [12]:
# [1] Import the scikit-learn metrics module for an accuracy calculation:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

# Print the confusion matrix.
print(confusion_matrix(y_test, y_pred))  

# [3a] Specify model accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# [3b] Specify model precision: what percentage of 
# positive tuples are labelled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# [3c] Specify model recall: how good is the model at 
# correctly predicting positive classes?
print("Recall:",metrics.recall_score(y_test, y_pred))



[[7360 3621]
 [ 789  587]]
Accuracy: 0.6431172614712308
Precision: 0.13949619771863117
Recall: 0.4265988372093023


The confusion matrix indicates (from left to right and top to bottom) there are:

7,360 true positives
3,621 false positives
789 false negatives
587 true negative. 
The accuracy of the model is 64%, precision is 14% and recall is 43%. Compare this output to the output of the BLR model? Can you see the difference? 

Yes, the BLR model predicted 91% fit, while the SVM predicted only 64% fit. So, which one can we trust? Next, you'll evaluate the accuracy of your BLR model by building an SVM, which will allow you to compare the two models. 